# Object Recognition Using Tensorflow Lite with MobilenetV2 Prebuild Model
**Full Coding At [Object_Recognition_(TensorFlow_Lite_Edition).py](Object_Recognition_(TensorFlow_Lite_Edition).py)**
<br>
*Libraries Used:* 
- tflite_runtime 
- OpenCV
- numpy 
- Picamera2 
- libcamera 
- os
- time 

## Installing Libraries @ Dependencies 
- Build-In Libraries 
    - Picamera2 
    - libcamera
    - os
    - time 

- User Install Libraries
    - tflite_runtime 
    - OpenCV 
    - numpy 

### First, let's install the user install libraries ! 
1. tflite_runtime 
    - Follow the link [Tensorflow.org](https://www.tensorflow.org/lite/guide/python)
2. OpenCV 
    - Follow the link [OpenCV.org](https://docs.opencv.org/4.x/d2/de6/tutorial_py_setup_in_ubuntu.html)
3. NumPy 
    - Follow the link [numpy.org](https://numpy.org/install/)


### Verify the installed libraries 
1. tflite_runtime 
    - In Python File type: <br>
    `import tflite_runtime.interpreter as tflite` 
    - Installation sucess if codeing run without error  
2. OpenCV 
    - In Python File type: <br>
    `import cv2` <br>
    `print(cv2.__version__)`
    - The coding above will print the version of opencv library installed  
3. NumPy 
    - In Python File type: <br>
    `import numpy as np`
    - Installation sucess if codeing run without error 

## Downloading The model Files 
The Object Detection Model used in this script is: MobilenetV2(tflite)
Download the object detection Model from github 
1. Open the terminal in Raspberry Pi 
2. Type the command bellow
- `git clone https://https://github.com/raspberrypi/picamera2`
This will download the Model from RaspberryPi's Picamera2 repository 

## Let's Start Coding ! 
1. Import all the libraries 
    - Tensorflow Lite 
    - OpenCV 
    - NumPy 
    - Picamera2 
    - libcamera
    - os
    - time 

In [ ]:
## Tensorflow Lite Library
import tflite_runtime.interpreter as tflite

## For Image processing 
import cv2
import numpy as np

## To capture the frames from the camera
from picamera2 import Picamera2
from libcamera import controls

## For validation of the model and label files 
import os

## To get the accurate time 
import time